In [ ]:
!nvidia-smi

In [ ]:
# clone the Darknet
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/openCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [ ]:
# configure Darknet network for training YOLO c3
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg # 배치 1을 64로 수정
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/ max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@classes=255@classes=18@' cfg/yolov3_training.cfg
!sed -i '689 s@classes=255@classes=18@' cfg/yolov3_training.cfg
!sed -i '776 s@classes=255@classes=18@' cfg/yolov3_training.cfg

In [ ]:
# create  folder on google drive so that we can save there the weights
!mkdir "/content/drive/MyDrive/yolov3"

mkdir: cannot create directory ‘/content/drive/MyDrive/yolov3’: File exists


In [ ]:
!echo "hand" > data/obj.names
!echo -e 'classes=1\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-25 23:03:09--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  11.1MB/s    in 34s     

2020-12-25 23:03:44 (4.51 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
# extract images
!unzip /content/drive/MyDrive/yolov3/images.zip -d data/obj

Archive:  /content/drive/MyDrive/yolov3/images.zip
  inflating: data/obj/question.ecf08d52-4204-11eb-a6c6-d0c6375f6cfb.jpg  
  inflating: data/obj/question.ecf08d52-4204-11eb-a6c6-d0c6375f6cfb.txt  
  inflating: data/obj/question.ee253d02-4204-11eb-96dc-d0c6375f6cfb.jpg  
  inflating: data/obj/question.ee253d02-4204-11eb-96dc-d0c6375f6cfb.txt  
  inflating: data/obj/question.ef613f8c-4204-11eb-9bd4-d0c6375f6cfb.jpg  
  inflating: data/obj/question.ef613f8c-4204-11eb-9bd4-d0c6375f6cfb.txt  
  inflating: data/obj/question.f09440ee-4204-11eb-a164-d0c6375f6cfb.jpg  
  inflating: data/obj/question.f09440ee-4204-11eb-a164-d0c6375f6cfb.txt  
  inflating: data/obj/question.f1c6deca-4204-11eb-a27f-d0c6375f6cfb.jpg  
  inflating: data/obj/question.f1c6deca-4204-11eb-a27f-d0c6375f6cfb.txt  
  inflating: data/obj/question.f2f9905e-4204-11eb-96b4-d0c6375f6cfb.jpg  
  inflating: data/obj/question.f2f9905e-4204-11eb-96b4-d0c6375f6cfb.txt  
  inflating: data/obj/question.f42c34dc-4204-11eb-9db3-d0c637

In [ ]:
import glob
images_list = glob.glob('data/obj/*.jpg')
print(images_list)

['data/obj/WIN_20201222_03_29_52_Pro.jpg', 'data/obj/WIN_20201222_03_33_01_Pro.jpg', 'data/obj/WIN_20201222_03_30_56_Pro.jpg', 'data/obj/question.f42c34dc-4204-11eb-9db3-d0c6375f6cfb.jpg', 'data/obj/WIN_20201222_03_31_02_Pro (2).jpg', 'data/obj/WIN_20201222_03_31_04_Pro (2).jpg', 'data/obj/WIN_20201222_03_36_24_Pro.jpg', 'data/obj/question.f55ee866-4204-11eb-b607-d0c6375f6cfb.jpg', 'data/obj/WIN_20201222_03_31_06_Pro (2).jpg', 'data/obj/WIN_20201222_03_31_05_Pro.jpg', 'data/obj/WIN_20201222_03_30_40_Pro.jpg', 'data/obj/WIN_20201222_03_30_58_Pro.jpg', 'data/obj/question.f1c6deca-4204-11eb-a27f-d0c6375f6cfb.jpg', 'data/obj/question.ecf08d52-4204-11eb-a6c6-d0c6375f6cfb.jpg', 'data/obj/WIN_20201222_03_36_26_Pro.jpg', 'data/obj/WIN_20201222_03_36_26_Pro (2).jpg', 'data/obj/WIN_20201222_03_30_51_Pro.jpg', 'data/obj/WIN_20201222_03_30_52_Pro.jpg', 'data/obj/WIN_20201222_03_31_00_Pro.jpg', 'data/obj/WIN_20201222_03_32_59_Pro.jpg', 'data/obj/WIN_20201222_03_33_00_Pro.jpg', 'data/obj/WIN_2020122

In [ ]:
# create training.txt file
file = open('data/train.txt','w')
file.write('\n'.join(images_list))
file.close()

In [ ]:
# start training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 88552, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.879046), count: 4, class_loss = 0.000113, iou_loss = 0.045153, total_loss = 0.045266 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 88556, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.889820), count: 3, class_loss = 0.000064, iou_loss = 0.030228, total_loss = 0.030292 
v3 (mse loss, Normaliz